# Query hash information

## Introduction

- UUID: **9b9f655d-100d-490e-b3b3-ec24cf5526bd**
- Started from [issue 15](https://github.com/MISP/misp-playbooks/issues/15)
- State: **Published**
- Purpose: This playbook is complementary to the playbooks for [static malware analysis](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_malware_triage.ipynb) and [dynamic malware analysis](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_malware_triage_upload_sample.ipynb) and investigates **file hashes**.
    - It checks if the hashes are found on **MISP warninglists**, in MISP events or in MISP feeds.
    - Uses the information from **VirusTotal**, **Hashlookup** and **MalwareBazaar** to provide context information on hashes.
    - It creates a **MISP report** for each hash.
    - Run this playbook if you do not have the sample and have to work with only the hash.
- Tags: [ "hash", "reputation", "abuse", "sample", "file", "malware" ]
- External resources: **Mattermost**, **VirusTotal**, **Hashlookup**, **MalwareBazaar**
- Target audience: **SOC**, **CSIRT**, **CTI**

# Playbook

- **Query hash information**
    - Introduction
- **Preparation**
    - PR:1 Initialise environment
    - PR:2 Verify MISP modules
    - PR:3 Load helper functions
    - PR:4 Set helper variables
    - PR:5 Provide the list of hashes
    - PR:6 MISP event details
    - PR:7 Setup MISP event link
- **Investigate**
    - IN:1 Context details for the hashes
    - IN:2 Add the hashes to the event
    - IN:3 Matches with MISP warninglists
    - IN:4 Download a sample    
- **Correlation**
    - CR:1 Correlation with MISP events
    - CR:2 Correlation with MISP feeds
- **Enrichment**
    - ER:1 Enrich with information from VirusTotal
    - ER:2 Enrich with information from Hashlookup
    - ER:3 Enrich with information from MalwareBazaar    
    - ER:4 Investigation and enrichment report
    - ER:5 PEinfo report
- **Summary**
    - EN:1 MISP indicators
    - EN:2 Create the summary of the playbook
    - EN:3 Print the summary
    - EN:4 Send a summary to Mattermost
    - EN:5 End of the playbook
- External references
- Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Indicate if PyMISP should attempt to verify the certificate or ignore errors
mattermost_playbook_user="<MATTERMOST USER>"
mattermost_hook="<MATTERMOST WEBHOOK>"
virustotal_apikey="<VIRUSTOTAL_APIKEY>"
```

In [ ]:
# Initialise Python environment
import urllib3
import sys
import json
import base64
import uuid
import time
import re
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
import requests
from pymisp import *
from pymisp.tools import GenericObjectGenerator
from abuse_finder import domain_abuse, ip_abuse, email_abuse, url_abuse
import textwrap
from datetime import datetime

# Used for -optionally- drawing the maps.
import folium

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
misp_headers = {"Authorization": misp_key,  "Content-Type": "application/json", "Accept": "application/json"}
print("I will use the MISP server \033[92m{}\033[90m for this playbook.\n".format(misp_url))

## PR:2 Verify MISP modules

This playbook uses the MISP modules to obtain additional correlation or enrichment information. [MISP modules](https://github.com/MISP/misp-modules) are autonomous modules that can be used to extend MISP for new services such as expansion, import and export. The modules are written in Python 3 following a simple API interface. The objective is to ease the extensions of MISP functionalities without modifying core components. The API is available via a simple REST API which is independent from MISP installation or configuration.

In the next cell we check if we have access to the **MISP module** server and if the required modules are enabled.

In [ ]:
# Where can we find the local MISP Module server? You can leave this to the default setting in most cases.
misp_modules_url = "http://127.0.0.1:6666"

# How long do we wait between queries when using the MISP modules (API rate limiting of external service such as VirusTotal)
misp_modules_wait = 3

# Initiliasation
misp_modules = {}
misp_modules_headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
misp_modules_in_use = ["hashlookup", "virustotal_public", "malwarebazaar"]
# Code block to query the MISP module server and check if our modules are enabled
res = requests.get("{}/modules".format(misp_modules_url), headers=misp_modules_headers)
for module in res.json():
    for module_requested in misp_modules_in_use:
        if module.get("name", False) == module_requested:
            misp_modules[module_requested] = {"enabled": True, "input": module.get("mispattributes").get("input")}
            print("Found the \033[92m{}\033[90m MISP module (Accepted input: {}).".format(module_requested, misp_modules[module_requested]["input"]))
print("\n")

## PR:3 Load helper functions

The next cell contains **helper functions** that are used in this playbook. 

Instead of distributing helper functions as separate Python files this playbook includes all the required code as one code cell. This makes portability of playbooks between instances easier. The downside is that functions defined in this playbook need to be defined again in other playbooks, which is not optimal for code re-use. For this iteration of playbooks it is chosen to include the code in the playbook (more portability), but you can easily create one "helper" file that contains all the helper code and then import that file in each playbook (for example by adding to the previous cell `from helpers import *`). Note that the graphical workflow image is included as an external image. A missing image would not influence the further progress of the playbook.

In [ ]:
def pb_get_misp_tags(tags=[], local_tags=[]):
    '''
    Get a list of MISP tags based on a Python list

    :param misp: MISP object
    :param object_template: which object template to return
    '''
    misp_tags = []
    for el in tags:
        t = MISPTag()
        t.name = el
        t.local = False
        misp_tags.append(t)

    for el in local_tags:
        t = MISPTag()
        t.name = el
        t.local = True
        misp_tags.append(t)
    return misp_tags



def get_playbook_result(value, source, category, res_type):
    '''
    Lookup the playbook results for a list of entries
    
    : param value: The IP address under investigation
    : param source: Which source provided the enrichment?
    : param category: Enrichment category
    : param res_type: Enrichment type
    '''
    return_value = []
    for el in playbook_results[value]:
        if el["source"] == source and el["category"] == category and el["type"] == res_type:
            if el["enriched"] not in return_value:
                if el["enriched"] is list:
                    for list_item in el["enriched"]:
                        return_value.append(str(list_item))
                else:
                    return_value.append(str(el["enriched"]))
    return_value = ' '.join(return_value).strip()
    if len(return_value) < 1:
        return_value = "-"
    return return_value



def pb_add_enrichment(playbook_results, field, entry, key, value):
    '''
    Add an enrichment (or correlation) entry but first check that the value is not already there
    
    : param playbook_results: all the enrichment results
    : param field
    : param entry
    : param key
    : param value
    '''
    skip_field = False
    for existing_entry in playbook_results.get(field, []):
        if existing_entry.get(key, False) == value:
            skip_field = True
            print(" Not adding to playbook results because of duplicate. Already added via {}".format(existing_entry.get("source", False)))
    if not skip_field:
        if field in playbook_results:
            playbook_results[field].append(entry)
        else:
            playbook_results[field] = [entry]
    return playbook_results



def calculate_risk_score(ratio):
    '''
    Calculate risk score and return the score and a tag
    '''
    detection_ratio_perc = 0
    score = 0
    base = 0
    tag = ""
    if len(ratio) > 0:
        for detection_ratio in ratio:
            score += int(detection_ratio.split("/")[0])
            base += int(detection_ratio.split("/")[1])
        if base > 0:
            detection_ratio_perc = round((score / base) * 100, 2)
            if detection_ratio_perc > 0:
                if detection_ratio_perc >= 75:
                    tag = "misp:threat-level=\"high-risk\""
                elif detection_ratio_perc >= 50:
                    tag = "misp:threat-level=\"medium-risk\""
                #elif detection_ratio_perc >= 25:
                else:
                    tag = "misp:threat-level=\"low-risk\""
            else:
                tag = "misp:threat-level=\"no-risk\""
    return detection_ratio_perc, tag



def resolve_hash_type(hash_type):
    '''
    Do an educated guess for the hash type
    '''
    if len(hash_type) in file_hash_type_length:
        return file_hash_type_length[len(hash_type)]
    else:
        return False

print("\033[92mHelper functions loaded\033[90m.\n".format(misp_url))

## PR:4 Set helper variables

This cell contains **helper variables** that are used in this playbook. Their usage is explained in the next steps of the playbook.

- `samples` : a dictionary of the objects that are created when the playbook progresses

In [ ]:
# Dictionary to playbook results and some of the core objects that are created
samples = {}

# Used as input for educated guesses on the hash type
file_hash_type_length = {32:"md5", 40:"sha1", 64:"sha256", 128:"sha512"}

# Hash types returned by external VT
file_hash_types = ["md5", "sha1", "sha256", "ssdeep"]

# A set of regular expressions that we use to determine the attribute type in a warninglist
regular_expressions = {"sha256": "^[a-fA-F0-9]{64}$",
                       "md5": "^[a-fA-F0-9]{32}$",
                       "hostname": "^[a-zA-Z0-9.\-_]+\.[a-zA-Z]{2,}$",
                       "sha1": "^[a-fA-F0-9]{40}$",
                       "url": "^(http|https):\/\/[-a-zA-Z0-9-]{2,256}\.[-a-zA-Z0-9-]{2,256}",
                       "ip-src": "[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}",
                       "ip-dst": "[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}"
                      }

## PR:5 Provide the list of hashes

Add the **hashes** that you want to investigate with this playbook to the variable `query`. You can provide one hash or a list of hashes. You can supply any type of hash, for example MD5, SHA1 or SHA256.

In [1]:
# Provide one or more hashes
query = []

In [ ]:
# Simple function to make a list. Makes it more consistent to work with the user input in the next cells.
if type(query) == str:
    query =[query]
print("The playbook will do the hash information query for \033[92m{}\033[90m\n".format(query))

## PR:6 MISP event details

### Event title

In this playbook we create a **new** MISP event with title **IP reputation _IPlist_**. You get the chance to override this default title but remember that it is good practice to choose a self-explanatory **event title**. This event title is shown in the MISP event index and should provide you the necessary information what the event is about. You should avoid using generic event titles. Read the [Best Practices in Threat Intelligence](https://www.misp-project.org/best-practices-in-threat-intelligence.html) for further guidance.

### Contexualisation

This playbook adds event contexualisation via the **tags** that are defined in `event_additional_global_tags` (for *global* tags) and `event_additional_local_tags` (for *local* tags). As a reminder, whereas *global* tags remain attached to the events that you share with your community, the *local* tags are not shared outside your organisation. It's also a good idea to primarily use tags that are part of a [taxonomy](https://github.com/MISP/misp-taxonomies), this allows you to make the contexualisation more portable accross multiple MISP instances.

In this playbook the list of tags is build via one of the helper functions `pb_get_misp_tags`. This function takes two arguments, first a list of tags to convert as *global* tags, and secondly a list of tags to convert as *local* tags. It then returns a Python list of MISPTag objects.

### Traffic Light Protocol

The default **TLP** for this event is **<span style='color:#FFBF00'>tlp:amber</span>**. The Traffic Light Protocol (TLP) facilitates sharing of potentially sensitive information and allows for more effective collaboration. TLP is a set of four standard labels to indicate the sharing boundaries to be applied by the recipients. TLP is always set by the creator of information. You can find more information at [FIRST](https://www.first.org/tlp/). You can specify the TLP via `event_tlp`.

### MISP Galaxies

This playbook can also add MISP galaxies to the event with the variable `event_galaxies`. You can also leave the list empty if you do not want to add galaxies in this stage of the investigation.

### MISP distribution, threat level and analysis level

Optionally you can specifiy a MISP **distribution** (with `event_distribution`), **threat level** (with `event_threat_level_id`) or **analysis state** (with `event_analysis`). The event **date** is set to today via `event_date`.

If you cannot remember the options for distribution, threat level or the analysis state then use the next cell to guide you. This cell is set as **raw**. If you **change its type to code** and execute the cell you get an overview of the options available for creating a MISP event.

## PR:7 Setup MISP event link

By default the playbook will generate a **title** with a prefix and the IPs you want to investigate. You can override this event title with the variable `event_title`. If you leave `event_title` empty the playbook will generate the MISP event title for you.

In [ ]:
# Provide the event title for a new event. Leave blank for the playbook to auto generate one
event_title = ""

# Prefix for auto generate event title
event_title_default_prefix = "Hash information"

# Optionally, you can change TLP, add additional event (local and global) tags, threatlevel, analysis state or distribution level
event_tlp = "tlp:amber"

# Event context
event_additional_global_tags = []                                 # This needs to be a Python list
event_additional_local_tags = ["workflow:state=\"incomplete\""]   # This needs to be a Python list

# Event galaxies
event_galaxies = ["misp-galaxy:mitre-attack-pattern=\"Malware - T1587.001\"", 
                  "misp-galaxy:mitre-attack-pattern=\"Malware - T1588.001\""]

# Additional MISP event settings
event_threat_level_id = ThreatLevel.low
event_analysis = Analysis.ongoing
event_distribution = Distribution.your_organisation_only
event_date = date.today()

### Create MISP event

The next cell **creates the MISP event** and stores the event object in `misp_event`.

In [ ]:
# Code block to create the event or add data to an existing event
event_title = event_title.strip()

if not(len(event_title) > 0):
    query_sample = ""
    for key in query:
        query_sample = "{} {}".format(key, query_sample)
    event_title = "{} for {}".format(event_title_default_prefix, query_sample)

# Construct the event tags
event_additional_global_tags.append(event_tlp)
if len(event_galaxies) > 0:
    event_additional_global_tags.append(event_galaxies)
event_tags = pb_get_misp_tags(event_additional_global_tags, event_additional_local_tags)

# Create the PyMISP object for an event
event = MISPEvent()
event.info = event_title
event.distribution = event_distribution
event.threat_level_id = event_threat_level_id
event.analysis = event_analysis
event.set_date(event_date)

# Create the MISP event on the server side
misp_event = misp.add_event(event, pythonify=True)
print("Continue the playbook with the new \033[92mcreated\033[90m MISP event ID {} with title \033[92m{}\033[90m and UUID {}".format(misp_event.id, misp_event.info, misp_event.uuid))
for tag in event_tags:
    if len(tag) > 0:
        misp.tag(misp_event.uuid, tag, local=tag.local)
        print(" \033[92mAdded\033[90m event tag {}".format(tag))

# Investigate

This section starts the investigation for the hash information.

## IN:1 Context details for the hashes

The first step sets the contextualisation (tags) for the hashes with `attribute_tags`. These tags are added to the MISP attributes.

In [ ]:
attribute_tags = pb_get_misp_tags(["PAP:GREEN", "course-of-action:passive=\"discover\"",
                                                  "course-of-action:passive=\"detect\""])

## IN:2 Add the hashes to the event

Next we add the hashes to the MISP event. The hashes are added as part of MISP `file` **objects**.

In [ ]:
print("Add hashes to MISP event.")
for hash_value in query:
    hash_type = resolve_hash_type(hash_value)
    if hash_type:
        file_object = MISPObject("file")
        file_object.add_attribute(hash_type, hash_value, tags=attribute_tags, comment="Added by playbook")
        response = misp.add_object(misp_event.uuid, file_object, pythonify=True)
        if not "errors" in response:
            samples[hash_value] = {}
            samples[hash_value]["hash_type"] = hash_type
            samples[hash_value]["fileobject"] = response
            samples[hash_value]["fileobject_uuid"] = response.uuid
    
            samples[hash_value][hash_type] = hash_value
            for attribute in response.attributes:
                if attribute.object_relation == hash_type:
                    samples[hash_value]["hash_attribute_uuid"] = attribute.uuid
                    break
                    
            samples[hash_value]["MISP"] = []
            samples[hash_value]["Feeds"] = []
            samples[hash_value]["detection_ratio"] = []
            samples[hash_value]["detection_ratio_indirect"] = []
            samples[hash_value]["vt"] = {}
            samples[hash_value]["warninglist"] = ""
            samples[hash_value]["malwarebazaar"] = {}
            samples[hash_value]["hashlookup"] = {}
            print(" Added file object for \033[92m{}\033[90m as {} with UUID {}".format(hash_value, hash_type, response.uuid))
        else:
            print(result)        
    else:
        print(" Unknown hash type \033[91m{}\033[90m".format(hash_value))

## IN:3 Matches with MISP warninglists

The [misp-warninglists](https://github.com/MISP/misp-warninglists) are lists of **well-known** indicators that can be associated to potential **false positives**, errors or mistakes. The warning lists are integrated in MISP to display an info/warning box at the event and attribute level if such indicators are available in one of the list. The lists are also used to filter potential false-positive at API level. MISP warninglists also serve a dual use. You cannot only use them to identify false positives, you can also use the lists to **track** if specific attributes (for example IPs or domains) are in the threat events you receive from your community.

Have a look at the other playbooks on warninglists, such as : [Create a custom MISP warninglist](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_create_custom_MISP_warninglist.ipynb) or [Retroscan with a MISP warninglist](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_retroscan_with_MISP_warninglist.ipynb), for more details.

In [ ]:
print("Search for matches with MISP warninglists")
for key, sample in samples.items():
    result = misp.values_in_warninglist(key)
    if not "errors" in result and len(result) > 0:
        print(" Match with warninglist for \033[92m{}\033[90m".format(key))
        sample["warninglist"] = "\n### MISP Warninglists\n\n"
        for match in result[key]:
            print("  Found in \033[92m{}\033[90m".format(match["name"]))
            sample["warninglist"] = "{}- Found in **{}**\n".format(sample["warninglist"], match["name"])
    else:
        print(" No match found for \033[91m{}\033[90m".format(key))
print("Finished searching")

## IN:4 Download a sample

This playbook does not include automatic downloading of malware samples from external sources. 

You can search for the samples in [VirusShare](https://virusshare.com/), or if your license permits it, query [VirusTotal](https://www.virustotal.com). If you download the sample you use the playbooks for [static malware analysis](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_malware_triage.ipynb) and [dynamic malware analysis](https://github.com/MISP/misp-playbooks/blob/main/misp-playbooks/pb_malware_triage_upload_sample.ipynb) to investigate it.

# Correlation

## CR:1 Correlation with MISP events

Search the **MISP server** for events that match with one of the hashes you specified in `query`. A summary is also shown at the end of the playbook.

Only **published** events (`correlation_published`) and attributes that have the **to_ids** flag (`correlation_to_ids`) are considered. You can limit the number of results (`correlation_limit`) and limit the search with tags (`correlation_match_tags`).

In [ ]:
# Only query for published MISP events
correlation_published = False

# Only consider those values that have the to_ids field set to True
correlation_to_ids = True

# Limit the returned results to 1000 attributes
correlation_limit = 1000

# Only return results corresponding with these tags
correlation_match_tags = ["tlp:amber", "tlp:white"]

In [ ]:
print("Search for correlating MISP events")
# Code block to query MISP and find the correlations
for key, sample in samples.items():
    value = key
    search_match = misp.search("attributes", to_ids=correlation_to_ids, value=value, tags=correlation_match_tags,
                                   published=correlation_published, limit=correlation_limit, pythonify=True)
    if len(search_match) > 0:        
        for attribute in search_match:
            if attribute.Event.id != misp_event.id:   # Skip the event we just created for this playbook
                print(" Found \033[92m{}\033[90m in \033[92m{}\033[90m ({})".format(attribute.value, attribute.Event.info, attribute.Event.id ))
                entry = {"source": "MISP", "category": attribute.category, "type": attribute.type, "event_id": attribute.Event.id, "event_info": attribute.Event.info}
                samples[key]["MISP"].append(entry)
print("Finished searching for correlations")

### MISP events correlation table

The correlation results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost and TheHive.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Category", "Type", "Event", "Event ID"]
table.align["Value"] = "l"
table.align["Category"] = "l"
table.align["Type"] = "l"
table.align["Event"] = "l"
table.align["Event ID"] = "l"
table._max_width = {"Event": 50}
for key, sample in samples.items():
    for match in sample["MISP"]:
        table.add_row([match["source"], key, match["category"], match["type"], match["event_info"], match["event_id"]])
print(table.get_string(sortby="Value"))
table_mispevents = table

## CR:2 Correlation with MISP feeds

Search the **MISP feeds** for events that match with one of the hashes you specified in `query`. The results highly depend on the feeds you have enabled. For good results, it's advised you enable at least
- **Threatfox**
- **MalwareBazaar**

Be sure to check if other [MISP feeds](https://www.misp-project.org/feeds/) provide valuable results for your environment as well. Note that the correlation lookup in the MISP feeds does not return the name of the MISP event, it returns the UUID of the event as title.

In [ ]:
print("Search in MISP feeds")
misp_cache_url = "{}/feeds/searchCaches/".format(misp_url)
match = False
for key, sample in samples.items():
    value = key
    # Instead of GET, use POST (https://github.com/MISP/MISP/issues/7478)
    cache_results = requests.post(misp_cache_url, headers=misp_headers, verify=misp_verifycert, json={"value": value})
    for result in cache_results.json():
        if "Feed" in result:
            match = True
            print(" Found \033[92m{}\033[90m in \033[92m{}\033[90m".format(value, result["Feed"]["name"]))
            for match in result["Feed"]["direct_urls"]:
                entry = {"source": "Feeds", "feed_name": result["Feed"]["name"], "match_url": match["url"]}
                samples[key]["Feeds"].append(entry)

print("Finished searching in MISP feeds")
if not match:
    print("\033[93mNo correlating information found in MISP feeds.")

### MISP feed correlations table

The correlation results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost and TheHive.

In [ ]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Feed", "URL"]
table.align["Value"] = "l"
table.align["Feed"] = "l"
table.align["Feed URL"] = "l"
table._max_width = {"Feed": 50}
for key, sample in samples.items():
    for match in sample["Feeds"]:
        table.add_row([match["source"], key, match["feed_name"], match["match_url"]])
print(table.get_string(sortby="Value"))
table_mispfeeds = table

# Enrichment

Next we enrich the hash information with results from external sources.

## ER:1 Enrich with information from VirusTotal

Next we use [VirusTotal](https://www.virustotal.com/gui/). First we use the MISP modules to query VirusTotal. The advantage of using the MISP modules is that it returns MISP attributes and objects that can then be easily added to the event.

After using the modules, we query VirusTotal directly (without using the module) to get the **threat classification**, **sandbox** results and the **Sigma** rules that triggered detection for the file hashes.

In [ ]:
# Loop through the hashes

# Use the modules to create MISP objects
# Then do a second query to get the hash details

scan_count = 0
module_name = "virustotal_public"
headers = {
    "accept": "application/json",
    "x-apikey": virustotal_apikey
}

print("Verifying samples with {}".format(module_name))
for key, sample in samples.items():
    attribute_type = sample["hash_type"]
    value = key
    module_comment = "From {} for {}".format(module_name, key)
    data = {
        "attribute": {
            "type": f"{attribute_type}",
            "uuid": str(uuid.uuid4()),
            "value": f"{value}",
        },
        "module": module_name,
        "config": {"apikey": virustotal_apikey}
    }
    print("Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
    result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
    if "results" in result.json() and len(result.json()["results"]) > 0:
        result_json = result.json()["results"]

        for misp_attribute in result_json.get("Attribute", []):
            misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
            created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
            if not "errors" in created_attribute:
                print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                misp.add_object_reference(sample.get("fileobject").add_reference(created_attribute.uuid, "related-to"))
            else:
                print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
        for misp_object in result_json.get("Object", []):
            misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
            if len(misp_object["Attribute"]) > 0:
                created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                if not "errors" in created_object:
                    print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                    misp.add_object_reference(sample.get("fileobject").add_reference(created_object.uuid, "related-to"))
                    if misp_object["name"] == "virustotal-report":
                        direct_vt_report = False
                        detection_ratio = False
                        for attribute in misp_object.get("Attribute", []):
                            # We cannot check for a permalink with the value
                            # Permalinks point to the SHA256, and the value can be MD5, SHA1 or others
                            #if attribute.get("object_relation") == "permalink" and value in attribute.get("value"):
                            if attribute.get("object_relation") == "permalink" and "virustotal.com/gui/file/" in attribute.get("value"):
                                direct_vt_report = True
                            if attribute.get("object_relation") == "detection-ratio":
                                detection_ratio = attribute.get("value")
                        if detection_ratio:
                            if direct_vt_report:
                                samples[key]["detection_ratio"].append(detection_ratio)
                                print(" Got detection ratio \033[92m{}\033[90m ".format(detection_ratio))
                            else:
                                samples[key]["detection_ratio_indirect"].append(detection_ratio)
                                print(" Got indirect detection ratio \033[92m{}\033[90m ".format(detection_ratio))
                else:
                    print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

        risk1, tag1 = calculate_risk_score(sample.get("detection_ratio"))
        risk2, tag2 = calculate_risk_score(sample.get("detection_ratio_indirect"))
        if len(tag1) > 0:
            misp.tag(sample.get("hash_attribute_uuid"), tag1)

        risk_rating = "vt-detection-ratio={}\nvt-detection-ratio-indirect={}".format(risk1, risk2)
        samples[key]["detection_ratio_perc"] = risk1
        samples[key]["detection_ratio_indirect_perc"] = risk2
        samples[key]["risk_rating"] = risk_rating
        sample.get("fileobject").add_attribute("text", risk_rating)
        misp.update_object(sample.get("fileobject"))

        #################################################################

        print(" Query VirusTotal directly to get file details")
        vt_url = "https://www.virustotal.com/api/v3/files/{}".format(value)
        result = requests.get(vt_url, headers=headers)
        if "data" in result.json() and len(result.json()["data"]) > 0:
            result_json = result.json()["data"]["attributes"]
            samples[key]["vt"]["data"] = result_json
            samples[key]["vt"]["popular_threat_classification"] = result_json.get("popular_threat_classification", [])
            samples[key]["vt"]["sigma_analysis_results"] = result_json.get("sigma_analysis_results", [])
            samples[key]["vt"]["sandbox_verdicts"] = result_json.get("sandbox_verdicts", [])
            samples[key]["vt"]["sandboxes"] = []
            samples[key]["vt"]["markdown"] = ""

            # Get the tags of our "core" hash
            attribute_tags = misp.get_attribute(sample["hash_attribute_uuid"], pythonify=True).tags
            #sample_hashes = { hash_type: key}
            for vt_hash_type in file_hash_types:
                if not sample.get(vt_hash_type, False) and result_json.get(vt_hash_type, False):
                    #if sample[hash_type] != vt_hash_type and result_json.get(vt_hash_type, False):
                    sample.get("fileobject").add_attribute(vt_hash_type, result_json.get(vt_hash_type), comment=module_comment, tags=attribute_tags)
                    #sample_hashes[vt_hash_type] = result_json.get(vt_hash_type)
                    sample[vt_hash_type] = result_json.get(vt_hash_type)
                    misp.update_object(sample.get("fileobject"))

            if result_json.get("first_submission_date", False):
                print("  Got \033[92mfirst submission date\033[90m")
                datetime_obj = datetime.utcfromtimestamp(result_json.get("first_submission_date"))
                formatted_time = datetime_obj.strftime('%Y-%m-%d %H:%M:%S UTC')
                samples[key]["vt"]["markdown"] = "{}- First submission date: **{}**\n".format(samples[key]["vt"]["markdown"], formatted_time)
                sample.get("fileobject").add_attribute("text", "First submission date: {}".format(formatted_time), comment=module_comment)
            if result_json.get("reputation", False):
                print("  Got \033[92mreputation\033[90m")
                samples[key]["vt"]["markdown"] = "{}- Reputation: **{}**\n".format(samples[key]["vt"]["markdown"], result_json.get("reputation"))
                sample.get("fileobject").add_attribute("text", "Reputation: {}".format(result_json.get("reputation")), comment=module_comment)
            if result_json.get("tags", False):
                print("  Got \033[92mtags\033[90m")
                samples[key]["vt"]["markdown"] = "{}- Tags: **{}**\n".format(samples[key]["vt"]["markdown"], result_json.get("tags"))
            if result_json.get("magic", False):
                print("  Got \033[92mfile magic\033[90m")
                samples[key]["vt"]["markdown"] = "{}- File magic: **{}**\n".format(samples[key]["vt"]["markdown"], result_json.get("magic"))
                sample.get("fileobject").add_attribute("text", "File magic: {}".format(result_json.get("magic")), comment=module_comment)                
            if result_json.get("type_extension", False):
                print("  Got \033[92mtype extension\033[90m")
                samples[key]["vt"]["markdown"] = "{}- Type extension: **{}**\n".format(samples[key]["vt"]["markdown"], result_json.get("type_extension"))
                sample.get("fileobject").add_attribute("text", "Type extension: {}".format(result_json.get("type_extension")), comment=module_comment)
            misp.update_object(sample.get("fileobject"))
                
            if len(samples[key]["vt"]["popular_threat_classification"]) > 0:
                samples[key]["vt"]["markdown"] = "{}\n\n#### Popular threat classifications\n".format(samples[key]["vt"]["markdown"])
                print("  Got \033[92mthreat classifications\033[90m")
                for el in samples[key]["vt"]["popular_threat_classification"]:
                    if el == "suggested_threat_label":
                        samples[key]["vt"]["suggested_threat_label"] = samples[key]["vt"]["popular_threat_classification"][el]
                    samples[key]["vt"]["markdown"] = "{}\n- {}: {}".format(samples[key]["vt"]["markdown"], el, samples[key]["vt"]["popular_threat_classification"][el])

            if len(samples[key]["vt"]["sigma_analysis_results"]) > 0:
                print("  Got \033[92msigma analysis results\033[90m")
                samples[key]["vt"]["markdown"] = "{}\n\n#### Sigma analysis results\n".format(samples[key]["vt"]["markdown"])
                for el in samples[key]["vt"]["sigma_analysis_results"]:
                    samples[key]["vt"]["markdown"] = "{}\n - {}".format(samples[key]["vt"]["markdown"], el["rule_title"])

            if len(samples[key]["vt"]["sandbox_verdicts"]) > 0:
                print("  Got \033[92msandbox verdicts\033[90m")
                samples[key]["vt"]["markdown"] = "{}\n\n#### Sandbox verdicts\n".format(samples[key]["vt"]["markdown"])
                for el in samples[key]["vt"]["sandbox_verdicts"]:
                    samples[key]["vt"]["sandboxes"].append(el)
                    samples[key]["vt"]["markdown"] = "{}\n - **{}**".format(samples[key]["vt"]["markdown"], el)
                    for el_sandbox in samples[key]["vt"]["sandbox_verdicts"][el]:
                        samples[key]["vt"]["markdown"] = "{}\n   - {}: {}".format(samples[key]["vt"]["markdown"], el_sandbox, samples[key]["vt"]["sandbox_verdicts"][el][el_sandbox])

            if result_json.get("pe_info", False):
                summary_pe = "## PEFile summary\n\n"
                print("  Using info from \033[92mpeinfo\033[90m")
                summary_pe += "## {}\n\n".format(key)

                pe_info = result_json.get("pe_info")
                if pe_info.get("imphash", False):
                    sample["imphash"] = pe_info.get("imphash", False)
                resource_language = []
                if pe_info.get("resource_details", False):
                    for resource in pe_info.get("resource_details"):
                        if resource["lang"] not in resource_language:
                            resource_language.append(resource["lang"])
                sample["resource_language"] = resource_language
                
                summary_pe += " - Imphash: {}\n".format(pe_info.get("imphash",""))
                summary_pe += " - Resource language: {}\n".format(resource_language)
                summary_pe += "\n\n"
                if pe_info.get("import_list", False):
                    summary_pe += "### Imports\n\n"
                    import_list = pe_info.get("import_list")
                    for import_library in import_list:
                        summary_pe += "**{}**\n".format(import_library["library_name"])
                        for import_function in import_library["imported_functions"]:
                            summary_pe += "- {}\n".format(import_function)
                        summary_pe += "\n\n"
                samples[key]["vt"]["summary_pe"] = summary_pe
                samples[key]["vt"]["summary_pe_raw"] = pe_info
        print(" Finished query VirusTotal directly")
    else:
        print("No results for \033[91m{}\033[90m.".format(value))
    scan_count += 1
    if scan_count > 5:
        print("Sleeping for {} seconds".format(misp_modules_wait))
        time.sleep(misp_modules_wait)
        scan_count = 0
print("Finished {}".format(module_name))

## ER:2 Enrich with information from Hashlookup

We use [CIRCL Hashlookup](https://www.circl.lu/services/hashlookup/) to identify if any of the hashes correspond with **known hashes**.

In [ ]:
# Loop through the hashes

module_name = "hashlookup"
print("Lookup the file in {}".format(module_name))
for key, sample in samples.items():
    attribute_type = sample["hash_type"]
    value = key
    module_comment = "From {} for {}".format(module_name, key)
    data = {
        "attribute": {
            "type": f"{attribute_type}",
            "uuid": str(uuid.uuid4()),
            "value": f"{value}",
        },
        "module": module_name,
        "config": {"custom_API": False}
    }
    print("Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
    result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
    if "results" in result.json() and len(result.json()["results"]) > 0:
        result_json = result.json()["results"]
        hashlookup_hit = False
        hashlookuptext = ""
        for misp_attribute in result_json.get("Attribute", []):
            misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
            created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
            if not "errors" in created_attribute:
                print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                misp.add_object_reference(sample.get("fileobject").add_reference(created_attribute.uuid, "related-to"))
            else:
                print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
        for misp_object in result_json.get("Object", []):
            misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
            if len(misp_object["Attribute"]) > 0:
                created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                if not "errors" in created_object:

                    print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                    misp.add_object_reference(sample.get("fileobject").add_reference(created_object.uuid, "related-to"))
                    for attribute in misp_object.get("Attribute", []):
                        if attribute["object_relation"] not in ["MD5", "SHA-1", "SHA-256", "SSDEEP"]:
                            hashlookuptext = "{}: {}\n{}".format(attribute["object_relation"], attribute["value"], hashlookuptext)
                        if attribute.get("object_relation") == "KnownMalicious":
                            hashlookup_hit = True
                else:
                    print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

        if hashlookup_hit:
            misp.tag(sample.get("hash_attribute_uuid"), "misp-workflow:analysis=\"known-file-hash\"")

        sample.get("fileobject").add_attribute("text", hashlookuptext)
        samples[key]["hashlookup"]["data"] = result_json
        samples[key]["hashlookup"]["markdown"] = hashlookuptext
        misp.update_object(sample.get("fileobject"))
    else:
        print("No results for \033[91m{}\033[90m.".format(value))
print("Finished {}".format(module_name))

## ER:3 Enrich with information from MalwareBazaar

Now we query [MalwareBazaar](https://bazaar.abuse.ch/). Similar as with VirusTotal, we use the MISP module to get useful attributes and objects. Afterwards we query MalwareBazaar directly to get more details such as vendor intel and tags.

In [ ]:
# Loop through the hashes

module_name = "malwarebazaar"
print("Lookup the file in {}".format(module_name))
for key, sample in samples.items():
    attribute_type = sample["hash_type"]
    value = key
    module_comment = "From {} for {}".format(module_name, key)
    data = {
        "attribute": {
            "type": f"{attribute_type}",
            "uuid": str(uuid.uuid4()),
            "value": f"{value}",
        },
        "module": module_name,
    }
    print("Query \033[92m{}\033[90m as \033[92m{}\033[90m".format(value, attribute_type))
    result = requests.post("{}/query".format(misp_modules_url), headers=misp_modules_headers, json=data)
    if "results" in result.json() and len(result.json()["results"]) > 0:
        result_json = result.json()["results"]
        for misp_attribute in result_json.get("Attribute", []):
            misp_attribute["comment"] = "{}{}".format(module_comment, misp_attribute.get("comment", ""))
            created_attribute = misp.add_attribute(misp_event.uuid, misp_attribute, pythonify=True)
            if not "errors" in created_attribute:
                print(" Got {} \033[92m{}\033[90m".format(misp_attribute["type"], misp_attribute["value"]))
                misp.add_object_reference(sample.get("fileobject").add_reference(created_attribute.uuid, "related-to"))
            else:
                print(" Unable to add {} \033[92m{}\033[90m to MISP event".format(misp_attribute["type"], misp_attribute["value"]))
        for misp_object in result_json.get("Object", []):
            misp_object["comment"] = "{}{}".format(module_comment, misp_object.get("comment", ""))
            if len(misp_object["Attribute"]) > 0:
                created_object = misp.add_object(misp_event.uuid, misp_object, pythonify=True)
                if not "errors" in created_object:
                    print(" Got \033[92m{}\033[90m ".format(misp_object["name"]))
                    misp.add_object_reference(sample.get("fileobject").add_reference(created_object.uuid, "related-to"))
                else:
                    print(" Unable to add \033[92m{}\033[90m to MISP event".format(misp_object["name"]))

        #################################################################

        print(" Query MalwareBazaar directly to get file details")
        url = "https://mb-api.abuse.ch/api/v1/"
        result = requests.post(url, data={"query": "get_info", "hash": value})
        if "data" in result.json() and len(result.json()["data"]) > 0:
            result_json = result.json()["data"][0]
            samples[key]["malwarebazaar"]["markdown"] = ""
            samples[key]["malwarebazaar"]["data"] = result_json
            samples[key]["malwarebazaar"]["tags"] = result_json.get("tags", "")
            samples[key]["malwarebazaar"]["signature"] = result_json.get("signature", "")

            if len(samples[key]["malwarebazaar"]["tags"]) > 0:
                samples[key]["malwarebazaar"]["markdown"] = "{}\n**Tags**\n".format(samples[key]["malwarebazaar"]["markdown"])
                for tag in samples[key]["malwarebazaar"]["tags"]:
                    samples[key]["malwarebazaar"]["markdown"] = "{}\n - {}\n".format(samples[key]["malwarebazaar"]["markdown"], tag)

            if len(result_json.get("vendor_intel", [])) > 0:
                if len(result_json.get("vendor_intel", []).get("Triage", [])) > 0:
                    samples[key]["malwarebazaar"]["markdown"] = "{}\n**Triage tags and signatures**\n".format(samples[key]["malwarebazaar"]["markdown"])
                    for tag in result_json.get("vendor_intel", []).get("Triage", []).get("tags", []):
                        samples[key]["malwarebazaar"]["markdown"] = "{}\n - {}\n".format(samples[key]["malwarebazaar"]["markdown"], tag)
                    for signature in result_json.get("vendor_intel", []).get("Triage", []).get("signatures", []):
                        samples[key]["malwarebazaar"]["markdown"] = "{}\n - {}\n".format(samples[key]["malwarebazaar"]["markdown"], signature["signature"])
        print(" Finished query MalwareBazaar directly")
    else:
        print("No results for \033[91m{}\033[90m.".format(value))
print("Finished {}".format(module_name))

## ER:4 Investigation and enrichment report

The next cell summarises the investigation and enrichment details into a **MISP report**. The report is attached to the MISP event, and also used at the end of the playbook to create the summary.

In [ ]:
current_date = datetime.now()
formatted_date = current_date.strftime("%Y-%m-%d")
summary_iv = "## Investigation report\n\n"
for key, sample in samples.items():
    summary_iv_key = ""
    summary_iv_key += "## Analysis for {}\n".format(key)
    summary_iv_key += "- Date: **{}**\n".format(formatted_date)
    summary_iv_key += "- MISP event: **{}** ({})\n".format(misp_event.info, misp_event.id)
    summary_iv_key += "- Hash type: **{}**\n".format(sample["hash_type"])
    for hash_type in file_hash_types:
        if sample.get(hash_type, False) and hash_type != sample["hash_type"]:
            summary_iv_key += "- Associated {}  **{}**\n".format(hash_type, sample.get(hash_type))
    if sample.get("imphash", False):
        summary_iv_key += "- Associated {}  **{}**\n".format("imphash", sample.get("imphash"))
    if sample.get("resource_language", False):
        summary_iv_key += "- Resource languages: **{}**\n".format(sample.get("resource_language"))
    summary_iv_key += "\n\n"
    
    if sample.get("detection_ratio", False):
        summary_iv_key += "### Risk\n"        
        summary_iv_key += " - Detection ratio: **{}**\n".format(sample["detection_ratio"])
        summary_iv_key += " - Detection ratio percentage: **{}**\n".format(sample["detection_ratio_perc"])
        summary_iv_key += " - Detection ratio indirect: {}\n".format(sample["detection_ratio_indirect"])
        summary_iv_key += " - Detection ratio indirect percentage: {}\n".format(sample["detection_ratio_indirect_perc"])
        if sample.get("risk_rating", False):
            summary_iv_key += " - Risk rating: **{}**\n".format(sample["risk_rating"])
        summary_iv_key += "\n\n"

    if len(sample["warninglist"]) > 0:
        summary_iv_key += sample["warninglist"]
        summary_iv_key += "\n\n"
        
    if sample["vt"].get("markdown"):        
        summary_iv_key += "### VirusTotal\n"
        summary_iv_key += sample["vt"]["markdown"]
        summary_iv_key += "\n\n"
    if sample["hashlookup"].get("markdown"):    
        summary_iv_key += "### Hashlookup\n"
        summary_iv_key += sample["hashlookup"]["markdown"]
        summary_iv_key += "\n\n"
    if sample["malwarebazaar"].get("markdown"):    
        summary_iv_key += "### MalwareBazaar\n"
        summary_iv_key += sample["malwarebazaar"]["markdown"]
        summary_iv_key += "\n\n"
    
    event_title = "Hash analysis - {}".format(key)
    print("Creating MISP report \033[92m{}\033[90m".format(event_title))
    chunk_size = 61500
    for i in range(0, len(summary_iv_key), chunk_size):
        chunk = summary_iv_key[i:i + chunk_size]
        event_report = MISPEventReport()
        event_title_edit = event_title
        if i > 0:
            event_title_edit = "{} ({} > {})".format(event_title, i, i + chunk_size)
        event_report.name = event_title_edit
        event_report.content = chunk
        result = misp.add_event_report(misp_event.id, event_report)
        if "EventReport" in result:
            print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", []).get("id", 0)))
        else:
            print("Failed to create report for \033[91m{}\033[90m.".format(event_title))
            
    summary_iv += summary_iv_key

## ER:5 PEinfo report

The previous enrichment with VirusTotal can also return information obtained via peinfo. During the enrichment the results are added to a Markdown blob which you can now add to a MISP event.

In [ ]:
for key, sample in samples.items():
    print("Is there peinfo for {}?".format(key))
    if sample["vt"].get("summary_pe") and len(sample["vt"]["summary_pe"]) > 0:
        print(" Found!")
        event_title = "PEFile analysis - {}".format(key)
        print(" Creating MISP report \033[92m{}\033[90m".format(event_title))
        chunk_size = 61500
        for i in range(0, len(sample["vt"]["summary_pe"]), chunk_size):
            chunk = sample["vt"]["summary_pe"][i:i + chunk_size]
            event_report = MISPEventReport()
            event_title_edit = event_title
            if i > 0:
                event_title_edit = "{} ({} > {})".format(event_title, i, i + chunk_size)
            event_report.name = event_title_edit
            event_report.content = chunk
            result = misp.add_event_report(misp_event.id, event_report)
            if "EventReport" in result:
                print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", []).get("id", 0)))
            else:
                print("Failed to create report for \033[91m{}\033[90m.".format(event_title))
    else:
        print(" No \033[91mpeinfo\033[90m found")

# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed in the playbook and can also be send to a chat channel. 

## EN:1 MISP indicators

The next section first **queries MISP for the indicators added to the MISP event** that is linked to the execution of this playbook.

The indicators are stored in the variable `indicator_table` (table format) and `indicator_raw_list` (in raw format) which is used in a later section to create the playbook summary.

In [ ]:
# Get all the indicators for our event and store this is in a table. We can also use this for the summary.
indicator_search = misp.search("attributes", uuid=misp_event.uuid, to_ids=True, pythonify=True)
indicator_raw_list = []
indicator_table = PrettyTable()
if len(indicator_search) > 0:
    indicator_table.field_names = ["Type", "Category", "Indicator", "Comment"]
    indicator_table.align["Type"] = "l"
    indicator_table.align["Category"] = "l"
    indicator_table.align["Indicator"] = "l"
    indicator_table.align["Comment"] = "l"
    indicator_table.border = True
    for indicator in indicator_search:
        if indicator.value not in indicator_raw_list:
            comment = indicator.comment
            if hasattr(indicator, 'object_relation'):
                object_ind = misp.get_object(indicator.object_id, pythonify=True)
                comment = "From '{}' object {} {}".format(object_ind.name, object_ind.comment.lower(), comment.lower())
            indicator_table.add_row([indicator.type, indicator.category, indicator.value, comment])
            indicator_raw_list.append(indicator.value)
    print("Got \033[92m{}\033[90m indicator(s) from the event \033[92m{}\033[90m ({}).\n".format(len(indicator_raw_list), misp_event.info, misp_event.id))
else:
    print("\033[93mNo indicators found in the event \033[92m{}\033[90m ({})".format(misp_event.info, misp_event.id))

### Raw list of MISP indicators

The indicators are now stored in `indicator_search` (as Python objects) and `indicator_raw_list` (in raw format, only the indicators). Execute the next cell to display them in a table format. The table is also included in the summary sent to Mattermost and TheHive.

In [ ]:
if len(indicator_raw_list) > 0:
    print(indicator_table.get_string(sortby="Type"))
    print("\n\nIndicator list in raw format:")
    print("---------------------------------------------------")
    for el in indicator_raw_list:
        print("{}".format(el))
    print("---------------------------------------------------")

## EN:2 Create the summary of the playbook

The next section creates a summary and stores the output in the variable `summary` in Markdown format. It also stores an intro text in the variable `intro`. These variables are later used when sending information to Mattermost or TheHive.

In [ ]:
summary = "# MISP Playbook summary\nMalware triage with MISP event: **{}** ({}/events/view/{}). ".format(misp_event.info, misp_url, misp_event.id)

summary += "\n"
summary += summary_iv
summary += "\n"
intro = summary

summary += "## Indicators\n\n"
summary += "### Indicators table\n\n"
if len(indicator_raw_list) > 0:
    indicator_table.set_style(MARKDOWN)
    summary += indicator_table.get_string(sortby="Type")
    summary += "\n\n\n"
    summary += "### Indicators in **raw format**\n\n"
    for indicator in indicator_raw_list:
        summary += "{}\n\n".format(indicator)
    summary += "\n"
else:
    summary += "There are no indicators"
summary += "\n\n"

summary += "## Correlations\n\n"
summary += "### MISP event matches\n\n"
table_mispevents.set_style(MARKDOWN)
summary += table_mispevents.get_string()
summary += "\n\n"

summary += "### MISP feed matches\n\n"
table_mispfeeds.set_style(MARKDOWN)
summary += table_mispfeeds.get_string()
summary += "\n\n"

summary += "\n\n"

print("The \033[92msummary\033[90m of the playbook is available.\n")

## EN:3 Print the summary

Apart from the full summary of the investigation, the malware triage summary provides the necessary input for the analyst. This summary was previously also added as a MISP report.

In [ ]:
# Comment the printing of the summary to avoid cluttering the playbook by accident
#display_markdown(summary, raw=True)

## EN:4 Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [ ]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [ ]:
message = False
if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

## EN:5 End of the playbook 

In [ ]:
print("\033[92m End of the playbook")


## External references <a name="extreferences"></a>

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)
- [MalwareBazaar](https://bazaar.abuse.ch/)
- [Hashlookup](https://www.circl.lu/services/hashlookup/)
- [VirusTotal](https://www.virustotal.com)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
pyfaup
chardet
PrettyTable
ipywidgets
mattermostdriver
folium

```

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```